In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import scipy.stats as stats

%matplotlib inline

In [2]:
np.random.seed(123)
npts = 1000

#Random points
x = np.array([np.random.rand(3) for _ in range(npts)])

def density(x):
    return 2700.

def rotation_matrix(random=True):
    thx = stats.norm(loc=np.pi/3, scale=0.05).rvs()
    thy = stats.norm(loc=-np.pi/5, scale=0.1).rvs()
    thz = stats.norm(loc=0, scale=0.2).rvs()
    
    Rx = np.array([[1,           0,            0],\
                   [0, np.cos(thx), -np.sin(thx)],\
                   [0, np.sin(thx),  np.cos(thx)]])
    
    Ry = np.array([[ np.cos(thy), 0, np.sin(thy)],\
                   [          0, 1,           0],\
                   [-np.sin(thy), 0, np.cos(thy)]])
    
    Rz = np.array([[np.cos(thz), -np.sin(thz), 0],\
                   [np.sin(thz),  np.cos(thz), 0],\
                   [          0,            0, 1]])
    
    return Rz.dot(Ry.dot(Rx))

def stress(x, t):
    principal = np.diag([2.0*x[0], 0.0, 0.0])*t
    Q = rotation_matrix()
    return Q.dot(principal).dot(Q.T)

def voigt(A):
    order = [(0, 0), (1, 1), (2, 2), (1, 2), (0, 2), (0, 1), (2, 1), (2, 0), (1, 0)]
    return np.array([A[i,j] for i,j in order])
    
import pickle
fn = "example_data.txt"
f = open(fn, 'w')



header  = "This file contains fake data that is not intended to be representative\n"
header += "of any true material. It exists merely to demonstrate the filter's\n"
header += "capabilities.\n"
header += "BEGIN DATA\n"
f.write(header)

time = np.linspace(0, 1, 11)
for t in time:
    f.write("t = {0}\n".format(t))
    for i, point in enumerate(tqdm(x)):
        line  = "MP, {0}, {1}, {2}, {3}".format(i, point[0], point[1], point[2])
        line += ", {0}".format(density(point))
        line += "".join([", {0}".format(s) for s in voigt(stress(point, t))])
        line += "\n"
        f.write(line)
f.close()